# Dynamic RNN Make Pratical

Modify from this [notebook](https://www.kaggle.com/winternguyen/predict-household-electric-power-using-lstms)

Download the dataset from this [link](https://www.kaggle.com/uciml/electric-power-consumption-data-set) and put this dataset under the `dataset` folder.

# Introduction:

In this Notebook, I try to learn and build the Long Short-Term Memory (LSTM) recurrent neural network to fit one third of data and then predict the rest of data.
    
Database information:
    
(1) date: Date in format dd/mm/yyyy

(2) time: time in format hh:mm:ss

(3) global_active_power: household global minute-averaged active power (in kilowatt)

(4) global_reactive_power: household global minute-averaged reactive power (in kilowatt)

(5) voltage: minute-averaged voltage (in volt)

(6) global_intensity: household global minute-averaged current intensity (in ampere)

(7) sub_metering_1: energy sub-metering No. 1 (in watt-hour of active energy). It corresponds to the kitchen, containing mainly a dishwasher, an oven and a microwave (hot plates are not electric but gas powered).

(8) sub_metering_2: energy sub-metering No. 2 (in watt-hour of active energy). It corresponds to the laundry room, containing a washing-machine, a tumble-drier, a refrigerator and a light.

(9) sub_metering_3: energy sub-metering No. 3 (in watt-hour of active energy). It corresponds to an electric water-heater and an air-conditioner.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

from typing import Union

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from IPython.display import display

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

dataset/household_power_consumption.txt
dataset/mmt-rte-45-server-1-no-dist-tx-test.zip
dataset/mmt-rte-45-server-1-no-dist-tx-train.zip
dataset/mmt-rte-45-server-1-no-dist-tx-testing-data/training-data/transaction-cpu-time-server-0.csv
dataset/mmt-rte-45-server-1-no-dist-tx-testing-data/training-data/transaction-dependencies.txt
dataset/mmt-rte-45-server-1-no-dist-tx-testing-data/training-data/transaction-diskio-count-server-0.csv
dataset/mmt-rte-45-server-1-no-dist-tx-testing-data/training-data/transaction-features.csv
dataset/mmt-rte-45-server-1-no-dist-tx-testing-data/training-data/transaction-latency-server-0.csv
dataset/mmt-rte-45-server-1-no-dist-tx-testing-data/training-data/transaction-networkin-size-server-0.csv
dataset/mmt-rte-45-server-1-no-dist-tx-testing-data/training-data/transaction-networkout-size-server-0.csv
dataset/mmt-rte-45-server-1-no-dist-tx/training-data/transaction-cpu-time-server-0.csv
dataset/mmt-rte-45-server-1-no-dist-tx/training-data/transaction-dependenc

# Import & cleaning data

Importing the txt file takes more time than that of csv file.

In [10]:
from cost_estimator import Loader, Preprocessor, RegressionModels, Task

def get_df(dataset_path: Union[str, list]):
    ou_name = 'OU2 - Initialize Thread'
    task_name = 'latency'
    features = ['System CPU Load', 'Process CPU Load',
                'System Load Average', 'Thread Active Count']
    sample_size = 10000

    task = Task(ou_name, task_name)
    if isinstance(dataset_path, str):
        dataset_path = [dataset_path]
        
    df_list = []

    for path in dataset_path:
        # sub_task_name = f'basic1-RTE-{i}'

        loader = Loader(path, server_count=1, n_jobs=8)
        df_features = loader.load_features_as_df(load_from_pkl=True)
        df_latencies = loader.load_latencies_as_df(load_from_pkl=True)
        dict_depend = loader.load_dependencies_as_dict()
        
        df_list.append((df_features, df_latencies, dict_depend))

        # print(f'\n\n---------------------- Basic1-RTE-{i} Latency ----------------------')
        psr = Preprocessor(df_features, df_latencies, ou_name)
        psr.drop_warmup_data()

        mean_before_filter = float(psr.get_label().mean())
        std_before_filter = float(psr.get_label().std())

        psr.drop_outlier_and_na(drop_alg='std')

        mean_after_filter = float(psr.get_label().mean())
        std_after_filter = float(psr.get_label().std())

        psr.sample_features(sample_size)
        psr.specify_features(features)
        
    return df_list

In [11]:
df_list = get_df(dataset_path=['dataset/mmt-rte-45-server-1-no-dist-tx/training-data'])

# display(df_features)
# display(df_latencies)
# print(dict_depend[100])

Load features from dataset/mmt-rte-45-server-1-no-dist-tx/training-data/transaction-features.pkl
Load label from dataset/mmt-rte-45-server-1-no-dist-tx/training-data/latency.pkl
Drop warmup size: 81808, Size before/after drop warmup data: 265500/183692
Drop na size: 0, Size before/after drop na data: 183692/183692
Drop outlier size: 23699, Size before/after drop outlier data: 183692/159993
Drop sample size: 159993, Size before/after sample: 159993/10000
current feature columns: ['System CPU Load', 'Process CPU Load', 'System Load Average', 'Thread Active Count']


In [32]:
def fix_NaN(df):
    df.isnull().sum()
    df = df.fillna(df.mean())
    df.isnull().sum()
    # return df

def make_dataset(df_features, df_latencies, dict_depend):
    df_start_time = df_features[['Transaction ID', 'Start Time']]
    df_total_time = df_latencies[['Total']]
    df_feature = df_latencies[['Is Distributed', 'Total', 'OU0 - Broadcast', 'OU0 - ROUTE', 'OU1 - Generate Plan', 'OU2 - Initialize Thread',
                               'OU3 - Acquire Locks', 'OU4 - Read from Local', 'OU5M - Read from Remote', 'OU6 - Execute Arithmetic Logic', 
                               'NonOU - Push to Remote', 'OU7 - Write to Local', 'OU8 - Commit']]
    
    max_depend_num = 0
    depends = []
    
    for k in dict_depend.keys():
        depend_num = len(dict_depend[k])
        if max_depend_num < depend_num:
            max_depend_num = depend_num
        
    for index, row in df_features.iterrows():
        # print(f"ID: {row['Transaction ID']} | Start: {row['Start Time']} | Total: {row['Total']}") 
        if dict_depend is not None:
            depend_txns = dict_depend.get(row['Transaction ID'], None)
            
#             If no dependent Txns, assign a empty list
            if depend_txns is None:
                depend_txns = []
                
            depend_txns_num = len(depend_txns)

            if depend_txns_num <= max_depend_num:
                depends.append(depend_txns + ([0] * (max_depend_num - depend_txns_num)))
            else:
#                 If exceed the maximum number of dependenet Txns, truncate
                depends.append(depend_txns[:max_depend_num])
    
    df_depends = pd.DataFrame(depends, columns=[f"dep-{i}" for i in range(1, max_depend_num + 1)])
    df_concat = pd.concat([df_start_time, df_feature, df_depends], axis=1)
    df_concat['Is Distributed'] = df_concat['Is Distributed'].astype(int)
    
    df_X = df_concat
    df_Y = pd.DataFrame(df_features['Start Time'] + df_total_time['Total'], columns=['End Time'])
    
    return df_X, df_Y

In [33]:
df_train_X, df_train_Y = make_dataset(df_features=df_list[0][0], df_latencies=df_list[0][1], dict_depend=df_list[0][2])

print(f"df_train_X shape: {df_train_X.shape}")
print(f"df_train_Y shape: {df_train_Y.shape}")
display(df_train_X)
display(df_train_Y)

df_train_X shape: (265500, 72)
df_train_Y shape: (265500, 1)


,Transaction ID,Start Time,Is Distributed,Total,OU0 - Broadcast,OU0 - ROUTE,OU1 - Generate Plan,OU2 - Initialize Thread,OU3 - Acquire Locks,OU4 - Read from Local,...,dep-48,dep-49,dep-50,dep-51,dep-52,dep-53,dep-54,dep-55,dep-56,dep-57
0,1,0,0,422772,27650,8166,3002,509,180,331502,...,0,0,0,0,0,0,0,0,0,0
1,2,84,0,268962,84840,48902,134,328,18,202749,...,0,0,0,0,0,0,0,0,0,0
2,3,95,0,378067,85504,43993,356,1247,91,267447,...,0,0,0,0,0,0,0,0,0,0
3,4,102,0,408310,86214,46013,343,319,83,278215,...,0,0,0,0,0,0,0,0,0,0
4,5,108,0,257324,86674,46520,87,1446,17,183418,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265495,265496,179945291,0,6535,84,71,9,4133,1926,107,...,0,0,0,0,0,0,0,0,0,0
265496,265497,179945293,0,9207,126,234,32,551,4440,1582,...,0,0,0,0,0,0,0,0,0,0
265497,265498,179946896,0,6374,68,63,16,11,5820,117,...,0,0,0,0,0,0,0,0,0,0
265498,265499,179946967,0,894,82,111,7,282,2,135,...,0,0,0,0,0,0,0,0,0,0


,End Time
0,422772
1,269046
2,378162
3,408412
4,257432
...,...
265495,179951826
265496,179954500
265497,179953270
265498,179947861


# Data Preparation and fitting

In [34]:
# from sklearn.preprocessing import MinMaxScaler

# values = df_resample.values
# scaler = MinMaxScaler(feature_range=(0, 1))
# scaled = scaler.fit_transform(values)
# reframed = series_to_supervised(scaled, 1, 1)
# r = list(range(df_resample.shape[1]+1, 2*df_resample.shape[1]))
# reframed.drop(reframed.columns[r], axis=1, inplace=True)
# reframed.head()


In [35]:
def reshape_dataset(seq_len: int, dataset: pd.DataFrame, seq_num: int=None):
    # Set the Length of Sequence
    seq_len = 200

    # Data spliting into train and test data series. Only 4000 first data points are selected for traing purpose.
    values = dataset.values
    n = values.shape[0]
    if seq_num is None:
        seq_num = n // seq_len
    series_n = seq_num * seq_len
    print(f"Dataset shape: {values.shape}, Length of Sequence: {seq_len}, Number of Sequence: {seq_num}, Number of Series: {series_n}")
    values_truncated = values[:series_n]

    # n_train_time = 4000
    # train = values_truncated[:n_train_time, :]
    # test = values_truncated[n_train_time:, :]
    # train_x, train_y = train[:, :-1], train[:, -1]
    # test_x, test_y = test[:, :-1], test[:, -1]

    # Shape: [number of sequences, sequence lengh, feature size]
    # train_x = train_x.reshape((train_x.shape[0] // seq_len, seq_len, train_x.shape[1]))
    # test_x = test_x.reshape((test_x.shape[0] // seq_len, seq_len, test_x.shape[1]))
    # train_y = train_y.reshape((train_y.shape[0] // seq_len, seq_len))
    # test_y = test_y.reshape((test_y.shape[0] // seq_len, seq_len))
    
    df_dataset = values_truncated.reshape((values_truncated.shape[0] // seq_len, seq_len, values_truncated.shape[1]))
    return df_dataset

In [36]:
seq_len = 200

train_x, train_y = reshape_dataset(seq_len=seq_len, dataset=df_train_X), reshape_dataset(seq_len=seq_len, dataset=df_train_Y)
test_x, test_y = reshape_dataset(seq_len=seq_len, dataset=df_train_X), reshape_dataset(seq_len=seq_len, dataset=df_train_Y)

Dataset shape: (265500, 72), Length of Sequence: 200, Number of Sequence: 1327, Number of Series: 265400
Dataset shape: (265500, 1), Length of Sequence: 200, Number of Sequence: 1327, Number of Series: 265400
Dataset shape: (265500, 72), Length of Sequence: 200, Number of Sequence: 1327, Number of Series: 265400
Dataset shape: (265500, 1), Length of Sequence: 200, Number of Sequence: 1327, Number of Series: 265400


In [37]:
print(f'train_x shape: {train_x.shape}')
print(f'train_y shape: {train_y.shape}')
print(f'test_x shape: {test_x.shape}')
print(f'test_y shape: {test_y.shape}')

train_x shape: (1327, 200, 72)
train_y shape: (1327, 200, 1)
test_x shape: (1327, 200, 72)
test_y shape: (1327, 200, 1)


<h3> LSTM model setting <h3>

(1) 100 neurons in the first visible layer

(2) dropout 10%

(3) 1 neuron in the output layer for predicting Global_active_power

(4) The input shape will be 1 time step with 7 features

(5) The mean_squared_error loss function and the efficient adam version of stochastic gradient descent

(6) The model will be fit for 50 training epochs with a batch size of 70.

In [38]:
import tensorflow as tf
from tensorflow.keras.layers import Dropout, Dense, RNN, Input
from tensorflow.nn import relu

import tensorflow.keras as keras

from sklearn.metrics import mean_squared_error,r2_score
import matplotlib.pyplot as plt
import numpy as np

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [39]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# Vanilla RNN

In [40]:
class VanillaRNNCell(keras.layers.Layer):
    def __init__(self, units, **kwargs):
        self.units = units
        self.state_size = units
        super(VanillaRNNCell, self).__init__(**kwargs)

    def build(self, input_shape):
        self.kernel = self.add_weight(shape=(input_shape[-1], self.units),
                                      initializer='uniform',
                                      name='kernel')
        self.recurrent_kernel = self.add_weight(
            shape=(self.units, self.units),
            initializer='uniform',
            name='recurrent_kernel')
        self.built = True

    def call(self, inputs, states):
        prev_output = states[0]
        h = tf.tensordot(inputs, self.kernel, axes=1)
        output = h + tf.tensordot(prev_output, self.recurrent_kernel, axes=1)
        return output, [output]
    
def getVanillaRNN(inputShape):
    rnnLayer = RNN(VanillaRNNCell(100), dynamic=True, return_sequences=True, return_state=False)

    inputs = Input(shape=inputShape)
    x = rnnLayer(inputs)
    x = Dropout(0.1)(x)
    outputs = Dense(1)(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.summary()

    return model

# Single Unit Dynamic RNN
Howeve, since the hidden state contains only 1 unit, the capacity is limited. The performance is quite poor.

In [41]:
# SIngle unit output RNN
class SingleUnitDynamicRNNCell(keras.layers.Layer):
    def __init__(self, units, rnn_units=100, memory_size=3, **kwargs):
        super(SingleUnitDynamicRNNCell, self).__init__(**kwargs)
        # Must have variables
        self.units = units
        self.state_size = tf.TensorShape([memory_size, units])

        # Custom variables
        self.rnn_units = rnn_units
        self.memory_size = memory_size

    def build(self, input_shape):
        self.kernel = self.add_weight(
            shape=(input_shape[-1], self.rnn_units),
            initializer='uniform',
            name='kernel')
        self.recurrent_kernel = self.add_weight(
            shape=(self.units, self.rnn_units),
            initializer='uniform',
            name='recurrent_kernel')
        self.scalar_dense = self.add_weight(
            shape=(self.rnn_units, self.units),
            initializer='uniform',
            name='scalar_dense')
        self.built = True
    
    def generate_indice(self, states):
        batch_size = states.shape[0]
        # Random select
        return tf.random.uniform((batch_size, 1), minval=0, maxval=self.memory_size, dtype=tf.dtypes.int32)

    def select_memory(self, states):
        batch_size = states.shape[0]
        indice = self.generate_indice(states)

        # The sequence number for Indices
        sequence = tf.range(start=0, limit=batch_size)
        sequence = tf.reshape(sequence, (sequence.shape[0], 1))

        # Indices for gather_nd
        gather_indice = tf.concat([sequence, indice], axis=1)

        # Grab the selected datas according to the indice
        selected_state = tf.gather_nd(states, gather_indice)
        return selected_state

    def call(self, inputs, states):
        prev_output = states[0]

        # Recurrent NN
        h = tf.tensordot(inputs, self.kernel, axes=1)
        selected_state = self.select_memory(states[0])
        x = relu(h + tf.tensordot(selected_state, self.recurrent_kernel, axes=1))
        output = tf.tensordot(x, self.scalar_dense, axes=1)

        # Concat new state
        new_state = tf.reshape(output, (output.shape[0], 1, output.shape[1]))
        output_states = tf.concat([new_state, prev_output], axis=1)
        output_states = output_states[:, 0:self.memory_size, :]

        return output, [output_states]

def getSingleUnitRNN(inputShape, memory_size):
    rnnLayer = RNN(SingleUnitDynamicRNNCell(1, rnn_units=100, memory_size=memory_size), dynamic=True, return_sequences=True, return_state=False)

    inputs = Input(shape=inputShape)
    outputs = rnnLayer(inputs)
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.summary()

    return model

# Multi-Units Dynamic RNN

In [54]:
class DynamicRNNCell(keras.layers.Layer):
    def __init__(self, units, memory_size=3, **kwargs):
        super(DynamicRNNCell, self).__init__(**kwargs)
        # Must have variables
        self.units = units
        self.state_size = tf.TensorShape([memory_size, units])

        # Custom variables
        self.memory_size = memory_size

    def build(self, input_shape):
        self.kernel = self.add_weight(shape=(input_shape[-1], self.units),
                                      initializer='uniform',
                                      name='kernel')
        self.recurrent_kernel = self.add_weight(
            shape=(self.units, self.units),
            initializer='uniform',
            name='recurrent_kernel')
        self.built = True
    
    def generate_indice(self, states):
        batch_size = states.shape[0]
        # Random select
        return tf.random.uniform((batch_size, 1), minval=0, maxval=self.memory_size, dtype=tf.dtypes.int32)

    def select_memory(self, states):
        batch_size = states.shape[0]
        indice = self.generate_indice(states)

        # The sequence number for Indices
        sequence = tf.range(start=0, limit=batch_size)
        sequence = tf.reshape(sequence, (sequence.shape[0], 1))

        # Indices for gather_nd
        gather_indice = tf.concat([sequence, indice], axis=1)

        # Grab the selected datas according to the indice
        selected_state = tf.gather_nd(states, gather_indice)
        return selected_state

    def call(self, inputs, states):
        prev_output = states[0]

        # Recurrent NN
        h = tf.tensordot(inputs, self.kernel, axes=1)
        selected_state = self.select_memory(states[0])
        output = h + tf.tensordot(selected_state, self.recurrent_kernel, axes=1)

        # Concat new state
        new_state = tf.reshape(output, (output.shape[0], 1, output.shape[1]))
        output_states = tf.concat([new_state, prev_output], axis=1)
        output_states = output_states[:, 0:self.memory_size, :]

        return output, [output_states]

def getModel(inputShape, memory_size):
    rnnLayer = RNN(DynamicRNNCell(100, memory_size=memory_size), dynamic=True, return_sequences=True, return_state=False)

    inputs = Input(shape=inputShape)
    x = rnnLayer(inputs)
    x = relu(x)
    x = Dense(100)(x)
    x = relu(x)
    x = Dropout(0.1)(x)
    outputs = Dense(1)(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.summary()

    return model

In [ ]:
import datetime

batch_size = 70
memory_size = 3

# model = getVanillaRNN((train_x.shape[1], train_x.shape[2]))
model = getModel((train_x.shape[1], train_x.shape[2]), memory_size=memory_size)
# model = getSingleUnitRNN((train_x.shape[1], train_x.shape[2]), memory_size=memory_size)

# Network fitting
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(train_x, train_y, 
                    epochs=50, batch_size=batch_size, 
                    validation_data=(test_x, test_y), verbose=2, 
                    callbacks=[tensorboard_callback])

# Loss history plot
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 200, 72)]         0         
_________________________________________________________________
rnn_4 (RNN)                  (None, 200, 3, 100)       0 (unused)
_________________________________________________________________
tf.nn.relu_2 (TFOpLambda)    (None, 200, 3, 100)       0         
_________________________________________________________________
dense_5 (Dense)              (None, 200, 3, 100)       10100     
_________________________________________________________________
tf.nn.relu_3 (TFOpLambda)    (None, 200, 3, 100)       0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 200, 3, 100)       0         
_________________________________________________________________
dense_6 (Dense)              (None, 200, 3, 1)         101 

2021-12-07 00:20:44.855520: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-07 00:20:44.855552: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-07 00:20:44.855674: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1666] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.
2021-12-07 00:20:44.855972: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-12-07 00:20:44.855986: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1757] function cupti_interface_->Finalize()failed with error CUPTI could not be loaded or symbol could not be found.
2021-12-07 00:20:45.630358: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-07 00:20:45.630401: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler 

19/19 - 13s - loss: 5423060857389056.0000 - val_loss: 1732956490039296.0000
Epoch 2/50
19/19 - 12s - loss: 1024539148419072.0000 - val_loss: 292798758649856.0000
Epoch 3/50
19/19 - 12s - loss: 196827345321984.0000 - val_loss: 107665854300160.0000
Epoch 4/50
19/19 - 12s - loss: 141273436520448.0000 - val_loss: 76101133008896.0000
Epoch 5/50
19/19 - 12s - loss: 125645040386048.0000 - val_loss: 68192261111808.0000
Epoch 6/50
19/19 - 12s - loss: 113610089037824.0000 - val_loss: 58246081544192.0000
Epoch 7/50
19/19 - 12s - loss: 100778513530880.0000 - val_loss: 48817655250944.0000
Epoch 8/50
19/19 - 12s - loss: 90346071523328.0000 - val_loss: 39569172987904.0000
Epoch 9/50
19/19 - 12s - loss: 80023411228672.0000 - val_loss: 30961752342528.0000
Epoch 10/50
19/19 - 12s - loss: 71493094473728.0000 - val_loss: 23531368742912.0000
Epoch 11/50
19/19 - 12s - loss: 63402797957120.0000 - val_loss: 16872510586880.0000
Epoch 12/50
19/19 - 12s - loss: 56713520611328.0000 - val_loss: 12287447400448.0000

Note that, we have resampled the database into hour, so, every time step is one hour. We try first to check the prediction in 500 hours.

In [ ]:
size = test_x.shape[2]

# Prediction test
yhat = model.predict(test_x)
yhat = yhat.reshape((-1, 1))
print(yhat.shape)
test_x_reshape = test_x.reshape((-1, size))
print(test_x_reshape.shape)

# invert scaling for prediction
inv_yhat = np.concatenate((yhat, test_x_reshape[:, 1-size:]), axis=1)
# inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]

# invert scaling for actual
test_y_reshape = test_y.reshape(-1, 1)
inv_y = np.concatenate((test_y_reshape, test_x_reshape[:, 1-size:]), axis=1)
# inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]

# calculate RMSE
rmse = np.sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

aa=[x for x in range(500)]
plt.figure(figsize=(25,10)) 
plt.plot(aa, inv_y[:500], marker='.', label="actual")
plt.plot(aa, inv_yhat[:500], 'r', label="prediction")
plt.ylabel(df_train_Y.columns[0], size=15)
plt.xlabel('Time step for first 500 hours', size=15)
plt.legend(fontsize=15)
plt.show()

# Reference

Tensorflow 2.0 APIs
- [tf.range](https://www.tensorflow.org/api_docs/python/tf/range)
- [tf.gather_nd](https://www.tensorflow.org/api_docs/python/tf/gather_nd)
- [tf.keras.layers.RNN](https://www.tensorflow.org/api_docs/python/tf/keras/layers/RNN)
- [tf.keras.layers.LSTM](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM)
  - Also contain the example code of `return_sequences` and `return_state`
- [tf.tensordot](https://www.tensorflow.org/api_docs/python/tf/tensordot)
- [Introduction to tensor slicing](https://www.tensorflow.org/guide/tensor_slicing)